In [1]:
import os
os.chdir("..")

# import midi_np_translation.midi2input as midi2input
import midi_np_translation.midi2input_v2 as midi2input
import numpy as np
import tqdm

In [4]:
input_output_directories = [
    ("input_midi/Jazz_ML_MIDI/Jazz Midi/", "preprocessed_dataset/Jazz_ML_MIDI/"), 
    ("input_midi/jazz_standards/", "preprocessed_dataset/jazz_standards/")
]

# if not os.path.exists("preprocessed_dataset/Jazz_ML_MIDI/"):
#     os.mkdir("preprocessed_dataset/Jazz_ML_MIDI/")
# if not os.path.exists("preprocessed_dataset/jazz_standards/"):
#     os.mkdir("preprocessed_dataset/jazz_standards/")

### run in sequence

In [3]:
def run_in_sequence():
    for input_midi_dir, output_np_dir in input_output_directories:
        for midi_file_name in os.listdir(input_midi_dir):
        # print(midi_file_name)
            if not midi_file_name.lower().endswith(".mid"):
                continue
            try:
                data_rows, bass_notes = midi2input.midi_to_input(input_midi_dir + midi_file_name)
            except AssertionError as e:
                print(f"'{midi_file_name}' AssertionError: ", end="")
                print(e)
            except Exception as e:
                print(f"'{midi_file_name}' Error: ", end="")
                print(e)
            else:
                np.save(output_np_dir + midi_file_name + ".npy", data_rows)
                np.save(output_np_dir + midi_file_name + ".ans.npy", bass_notes)

# run_in_sequence()

### run in parallel

In [2]:
def process_a_single_midi_file(args):
    input_midi_dir, output_np_dir, midi_file_name = args
    if not midi_file_name.lower().endswith(".mid"):
        return
    try:
        data_rows, bass_notes = midi2input.midi_to_input(input_midi_dir + midi_file_name)
    except AssertionError as e:
        print(f"AssertionError '{midi_file_name}': ", end="")
        print(e)
    except Exception as e:
        print(f"Error '{midi_file_name}': ", end="")
        print(e)
    else:
        np.save(output_np_dir + midi_file_name + ".npy", data_rows)
        np.save(output_np_dir + midi_file_name + ".ans.npy", bass_notes)
        print(len(os.listdir(output_np_dir)))

In [4]:
import multiprocess

def run_in_parallel(input_output_directories):
    all_args = []
    for input_midi_dir, output_np_dir in input_output_directories:
        for midi_file_name in sorted(os.listdir(input_midi_dir)):
            all_args.append((input_midi_dir, output_np_dir, midi_file_name))

    pool = multiprocess.Pool(2)
    _ = pool.map(process_a_single_midi_file, all_args)

In [6]:
# input_output_directories = [("input_midi/Jazz_ML_MIDI/Jazz Midi/", "preprocessed_dataset/Jazz_ML_MIDI/"), 
#                             ("input_midi/jazz_standards/", "preprocessed_dataset/jazz_standards/")]
# input_output_directories = [("input_midi/irealpro_transposed/", "preprocessed_dataset/irealpro_dataset_v2/")]
input_output_directories = [("input_midi/midkar/", "preprocessed_dataset/midkar_v2/")]
run_in_parallel(input_output_directories)

2
4
6
8
10
12
14
16
AssertionError 'bill_bailey-2_rw.mid': didn't find bass instruments in midi file
18
20
22
24
26
28
AssertionError 'a_house_is_not_a_home_dm.mid': didn't find bass instruments in midi file
AssertionError 'a_lovely_way_to_spend_an_evening-1943-Vs2-kar_jpp.mid': didn't find bass instruments in midi file
30
32
34
36
38
40
Error 'a_taste_of_honey_jc2.mid': MIDI file has a largest tick of 11839910, it is likely corrupt
AssertionError 'a_taste_of_honey_ps2_lk.mid': didn't find bass instruments in midi file
AssertionError 'a_time_for_love_dm.mid': didn't find bass instruments in midi file
42
44
46
50
50
52
54
56
58
60
62
64
66
68
70


/usr/local/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


72
74
76
78
80
AssertionError 'blue_moon_1934_kar_jpp.mid': didn't find bass instruments in midi file
82
AssertionError 'aint_misbehavin_rj.mid': didn't find bass instruments in midi file
AssertionError 'aint_misbehavin_sk.mid': didn't find bass instruments in midi file
84
86
88
AssertionError 'blue_tango_gp.mid': didn't find bass instruments in midi file
90
92
AssertionError 'blues_in_f_sharp_bm.mid': didn't find bass instruments in midi file
94
96
98
100
Error 'bobsled_asw.mid': index 10330 is out of bounds for axis 0 with size 10330
102
104
Error 'body_and_soul_oz.mid': list index out of range
106
108
110
AssertionError 'all_of_me-1931-vs2-kar_jpp.mid': didn't find bass instruments in midi file
112
114
116
118
120
122
AssertionError 'all_the_things_you_are-1940-Kern-Mayerl_jpp.mid': didn't find bass instruments in midi file
124
126
128
AssertionError 'all_the_things_you_are_bm.mid': didn't find bass instruments in midi file
130
AssertionError 'all_the_way-1957-vs2-kar_jpp.mid': didn

/usr/local/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


246
248
250
252
AssertionError 'bazzas_boogie_bz2_bz3.mid': didn't find bass instruments in midi file
AssertionError 'cathedral_in_a_suitcase-Metheny_mg.mid': didn't find bass instruments in midi file
254
256
260
260
262
264
AssertionError 'because_of_you-kar_ea.mid': didn't find bass instruments in midi file
AssertionError 'because_of_you_1950-kar_jpp.mid': didn't find bass instruments in midi file
266
268
270
272
274
AssertionError 'behind_the_waterfall_kk.mid': didn't find bass instruments in midi file
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
AssertionError 'cry_baby_cry-1937-1-kar_jpp.mid': didn't find bass instruments in midi file
324
326
328
330
332
334
AssertionError 'deep_purple-ps_dm.mid': didn't find bass instruments in midi file
336
AssertionError 'deep_purple_da.mid': didn't find bass instruments in midi file
338
340
342
AssertionError 'didnt_we-dk633k-1_dk.mid': didn't find bass instruments in midi file
344
AssertionEr